In [2]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import requests
import datetime
import re
import json
from datetime import date, timedelta
from datetime import datetime
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,array, expr,first, last,when, split, col,lit, concat, date_format,to_utc_timestamp,to_timestamp,from_utc_timestamp, regexp_replace,posexplode_outer
client = bigquery.Client(location="us-central1")
print("Client creating using default project: {}".format(client.project))
pd.set_option('display.width', 1000)
pd.set_option("max_colwidth",10000)
pd.set_option("max_rows",1000)
pd.set_option("max_columns",100)

spark = SparkSession.builder.appName("YT1-allocation-detailed").getOrCreate()

Client creating using default project: saas-analytics-io


In [5]:
environment = 'YT1'

In [6]:
if environment == 'YT1':
    project_number = '260718309650'

In [7]:
GCP_cost_table = spark.read.format('bigquery')\
.option('project','saas-analytics-io')\
.option('table','finance.gcp_billing_export_resource_v1_01FE23_B1D2D8_10D34D')\
.option("filter","JSON_VALUE(project.number)= '%s' and date(usage_start_time) between '2024-05-02' and '2024-07-01' "%(project_number))\
.load()

In [8]:
GCP_cost_table = GCP_cost_table.select(col("service.description").alias('service_description'),
                                          col("sku.description").alias('sku_description'),
                                          col("usage_start_time"),
                                          col("service.description").alias('id_project'),
                                          col("cost"),
                                          col("usage.amount").alias('usage_amount'),
                                          col("usage.unit").alias('usage_unit'),
                                          col("credits")[0].alias('credits_1'),
                                          col("credits")[1].alias('credits_2'),
                                          col("cost_type"),
                                          col("resource.name").alias('resource_name'),
                                          col("resource.global_name").alias('resource_global_name'),
                                         )   

GCP_cost_table = GCP_cost_table.select(col('service_description'),
                                          col('sku_description'),
                                          col("usage_start_time"),
                                          col('id_project'),
                                          col("cost"),
                                          col('usage_amount'),
                                          col('usage_unit'),
                                          col("credits_1.name").alias('credits_1_name'),
                                          col("credits_1.amount").alias('credits_1_amount'),
                                          col("credits_1.type").alias('credits_1_type'),
                                          col("credits_2.name").alias('credits_2_name'),
                                          col("credits_2.amount").alias('credits_2_amount'),
                                          col("credits_2.type").alias('credits_2_type'),
                                          col("cost_type"),
                                          col('resource_name'),
                                          col('resource_global_name'),
                                         ) 
GCP_cost_table.show()

+-------------------+--------------------+-------------------+--------------+--------+--------------------+------------+--------------------+----------------+--------------------+--------------+----------------+--------------+---------+--------------------+--------------------+
|service_description|     sku_description|   usage_start_time|    id_project|    cost|        usage_amount|  usage_unit|      credits_1_name|credits_1_amount|      credits_1_type|credits_2_name|credits_2_amount|credits_2_type|cost_type|       resource_name|resource_global_name|
+-------------------+--------------------+-------------------+--------------+--------+--------------------+------------+--------------------+----------------+--------------------+--------------+----------------+--------------+---------+--------------------+--------------------+
|          Cloud Run|CPU Allocation Ti...|2024-06-14 17:00:00|     Cloud Run|0.246574|        13698.507827|     seconds|                null|            null|     

In [9]:
GCP_cost_table = GCP_cost_table.withColumn("component",when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard%')), 'Mongo DB Platform - Shard')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-vm%')),'Mongo DB Platform - Shard')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-router%')),'Mongo DB Platform - Router')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-config%')),'Mongo DB Platform - config')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-analytics%')),'Mongo DB Analytics')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-temp%')),'Mongo DB Platform - Temp')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%kafka%')),'Kafka Browser')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%gke%')),'Containers (gke)')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%report%')),'Reports')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%nfs%')),'NFS')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%rabbit%')),'Rabbit')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%sftp%')), 'SFTP')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%vpn%')),'VPN')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%manager%')),'Manager')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%postgres%')),'Postgres')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%powerbi%')),'Power BI')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('ytem%')),'Microservices') 
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('monitoring%')),'Monitoring') 
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('analytics%')),'Microservices Analytics')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%vpc--connector%')),'VPC Connector')
.when((GCP_cost_table['resource_global_name'].like('%forwarding_rule%')),'Forwarding Rule')
.when((GCP_cost_table['resource_global_name'].like('%cloudfunctions%')),'Cloud Functions')
.when((GCP_cost_table['resource_global_name'].like('%scxyt1cloudsql%')),'Cloud SQL')
.when((GCP_cost_table['resource_global_name'].like('%ytemyt1cloudsql%')),'Cloud SQL')
.when((GCP_cost_table['resource_global_name'].like('%cloudsql%')),'Cloud SQL')
.when((GCP_cost_table['resource_global_name'].like('%k8s-mojix-io%')),'Containers (gke)')
.when((GCP_cost_table['resource_global_name'].like('%bigquery%')),'BigQuery')
.when((GCP_cost_table['resource_global_name'].like('%run.googleapis%')),'Run Google APIs')
.when((GCP_cost_table['resource_global_name'].like('%compute.googleapis%')),'Compute Google APIs')
.when((GCP_cost_table['resource_global_name'].like('%firestore.googleapis%')),'Firestore')
.when((GCP_cost_table['resource_global_name'].like('%dataflow.googleapis.com%')),'Dataflow')
.when((GCP_cost_table['resource_global_name'].like('%storage.googleapis.com%')),'Cloud Storage')
                                          )

In [10]:
GCP_cost_table.groupBy('component').sum().show()

+--------------------+--------------------+--------------------+---------------------+---------------------+
|           component|           sum(cost)|   sum(usage_amount)|sum(credits_1_amount)|sum(credits_2_amount)|
+--------------------+--------------------+--------------------+---------------------+---------------------+
|          Monitoring|           20.424458|1.448716520757657...|                 null|                 null|
|                null|  2411.7951910000006|5.042274089649408...|   -436.8767950000001|                 null|
|Mongo DB Platform...|   473.4418069999996|8.950190361490361...|   -84.50292099999996|                 null|
|                 VPN|   79.21118900000006|1.048258836465345...|           -20.803375|                 null|
|Mongo DB Platform...|   3578.738716999999|3.155887658638946...|   -676.0288440000004|                 null|
|Mongo DB Platform...|  51923.509564999484|7.586011944419253E20|  -3605.4873920000014|                 null|
|       Kafka Brows

In [11]:
GCP_cost_table_analysis = GCP_cost_table.filter((GCP_cost_table.component.isNull()))
GCP_cost_table_analysis.groupBy(['service_description','resource_name','resource_global_name']).sum().orderBy(col("sum(cost)").desc()).show(truncate=False)
#GCP_cost_table_analysis.groupBy(['resource_name','resource_global_name']).sum().orderBy(col("sum(cost)").desc()).show(truncate=False)

+--------------------+-------------+------------------------------------------------------------------------------------------+--------------------+----------------------+---------------------+---------------------+
|service_description |resource_name|resource_global_name                                                                      |sum(cost)           |sum(usage_amount)     |sum(credits_1_amount)|sum(credits_2_amount)|
+--------------------+-------------+------------------------------------------------------------------------------------------+--------------------+----------------------+---------------------+---------------------+
|Networking          |null         |null                                                                                      |1032.851532         |1.905335103793E12     |-436.8497949999998   |null                 |
|Cloud Logging       |null         |//logging.googleapis.com/projects/260718309650/locations/global/buckets/ytem-logs         |551.58920

In [22]:
GCP_cost_table = GCP_cost_table.withColumn("SubComponent",when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-1%')),'Mongo DB Platform - Shard 1')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-2%')),'Mongo DB Platform - Shard 2')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-3%')),'Mongo DB Platform - Shard 3')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-4%')),'Mongo DB Platform - Shard 4')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-5%')),'Mongo DB Platform - Shard 5')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-6%')),'Mongo DB Platform - Shard 6')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-7%')),'Mongo DB Platform - Shard 7')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-01%')),'Mongo DB Platform - Shard 1')       
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-02%')),'Mongo DB Platform - Shard 2')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-03%')),'Mongo DB Platform - Shard 3')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-04%')),'Mongo DB Platform - Shard 4')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-05%')),'Mongo DB Platform - Shard 5')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-06%')),'Mongo DB Platform - Shard 6')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-07%')),'Mongo DB Platform - Shard 7')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-analytics-01%')),'Mongo DB Analytics - Shard 1')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-analytics-02%')),'Mongo DB Analytics - Shard 2')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-vm002%')),'Mongo DB Platform - Shard 0')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-vm001%')),'Mongo DB Platform - Shard 0')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-vm003%')),'Mongo DB Platform - Shard 0')                                          
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('ytem%')),col('resource_name'))
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('analytics%')),col('resource_name'))
.otherwise('N/A'))

In [23]:
GCP_cost_table = GCP_cost_table.withColumn('usage_start_time_LOS_ANGELES', from_utc_timestamp(col("usage_start_time"),"America/Los_Angeles"))


In [24]:
GCP_cost_table = GCP_cost_table.na.fill(value=0,subset=["cost",'credits_1_amount','credits_2_amount'])
GCP_cost_table = GCP_cost_table.na.fill(value='N/A',subset=['SubComponent'])

In [25]:
GCP_cost_table = GCP_cost_table.withColumn("credits_discounts",when((GCP_cost_table['credits_1_type'].like('%DISCOUNT%'))&
                                                                    ((~(GCP_cost_table['credits_2_type'].like('%DISCOUNT%')))|
                                                                     (GCP_cost_table['credits_2_type'].isNull())),GCP_cost_table['credits_1_amount'])
                                          .when(((~(GCP_cost_table['credits_1_type'].like('%DISCOUNT%')))
                                                |(GCP_cost_table['credits_1_type'].isNull()))&
                                                (GCP_cost_table['credits_2_type'].like('%DISCOUNT%')),GCP_cost_table['credits_2_amount'])
                                          .when((GCP_cost_table['credits_1_type'].like('%DISCOUNT%'))&
                                                (GCP_cost_table['credits_2_type'].like('%DISCOUNT%')),GCP_cost_table['credits_1_amount']+GCP_cost_table['credits_2_amount']))

GCP_cost_table = GCP_cost_table.withColumn("credits_promotion",when((GCP_cost_table['credits_1_type'].like('%PROMOTION%'))&
                                                                    ((~(GCP_cost_table['credits_2_type'].like('%PROMOTION%')))|
                                                                     (GCP_cost_table['credits_2_type'].isNull())),GCP_cost_table['credits_1_amount'])
                                          .when(((~(GCP_cost_table['credits_1_type'].like('%PROMOTION%')))
                                                |(GCP_cost_table['credits_1_type'].isNull()))&
                                                (GCP_cost_table['credits_2_type'].like('%PROMOTION%')),GCP_cost_table['credits_2_amount'])
                                          .when((GCP_cost_table['credits_1_type'].like('%PROMOTION%'))&
                                                (GCP_cost_table['credits_2_type'].like('%PROMOTION%')),GCP_cost_table['credits_1_amount']+GCP_cost_table['credits_2_amount']))
#GCP_cost_table = GCP_cost_table.withColumn("credits_discounts",when(GCP_cost_table['credits_2_type'].like('%DISCOUNT%'),GCP_cost_table['credits_discounts']+GCP_cost_table['credits_2_amount']))
#GCP_cost_table = GCP_cost_table.withColumn("discounts_c2",when(GCP_cost_table['credits_2_type'].like('%DISCOUNT%'),GCP_cost_table['credits_1_amount']))
#GCP_cost_table = GCP_cost_table.withColumn("discounts_c2",when(GCP_cost_table['credits_2_type'].like('%DISCOUNT%'),GCP_cost_table['credits_1_amount']))


In [26]:
GCP_cost_table = GCP_cost_table.na.fill(value=0,subset=["credits_discounts",'credits_promotion'])

In [27]:
GCP_cost_table = GCP_cost_table.withColumn("storage_cost",when(GCP_cost_table['sku_description'].like('%SSD%'),GCP_cost_table['cost']))
GCP_cost_table = GCP_cost_table.withColumn("compute_cost",when((~(GCP_cost_table['sku_description'].like('%SSD%'))),GCP_cost_table['cost']))
GCP_cost_table = GCP_cost_table.withColumn('total_after_discounts', GCP_cost_table['cost'] + GCP_cost_table['credits_discounts'])
GCP_cost_table = GCP_cost_table.withColumn('total_after_discounts_promotion', GCP_cost_table['cost'] + GCP_cost_table['credits_discounts']+ GCP_cost_table['credits_promotion'])


#GCP_cost_table = GCP_cost_table.withColumn('total_after_discounts',GCP_cost_table['cost'] + GCP_cost_table['credits_1_amount']+ GCP_cost_table['credits_2_amount'])
#GCP_cost_table = GCP_cost_table.withColumn('amount_credits', GCP_cost_table['credits_1_amount']+ GCP_cost_table['credits_2_amount'])
   
    #GCP_cost_table.loc[(~(GCP_cost_table['sku_description'].str.contains('SSD'))),'processing_discount']=  GCP_cost_table['amount_credits_1']+GCP_cost_table['amount_credits_2']
    #GCP_cost_table.loc[(GCP_cost_table['sku_description'].str.contains('SSD')),'storage_discount']=  GCP_cost_table['amount_credits_1']+GCP_cost_table['amount_credits_2']
    #GCP_cost_table.loc[(~(GCP_cost_table['sku_description'].str.contains('SSD'))),'total_processing_after_discount']=  GCP_cost_table['cost'] + GCP_cost_table['amount_credits_1']+ GCP_cost_table['amount_credits_2']


In [28]:
GCP_cost_table.groupBy('credits_1_type').sum('credits_1_amount').show()

+--------------------+---------------------+
|      credits_1_type|sum(credits_1_amount)|
+--------------------+---------------------+
|SUSTAINED_USAGE_D...|  -10122.100980000007|
|                null|                  0.0|
|            DISCOUNT|  -469.71359100000006|
+--------------------+---------------------+



In [29]:
GCP_cost_table.printSchema()

root
 |-- service_description: string (nullable = true)
 |-- sku_description: string (nullable = true)
 |-- usage_start_time: timestamp (nullable = true)
 |-- id_project: string (nullable = true)
 |-- cost: double (nullable = false)
 |-- usage_amount: double (nullable = true)
 |-- usage_unit: string (nullable = true)
 |-- credits_1_name: string (nullable = true)
 |-- credits_1_amount: double (nullable = false)
 |-- credits_1_type: string (nullable = true)
 |-- credits_2_name: string (nullable = true)
 |-- credits_2_amount: double (nullable = false)
 |-- credits_2_type: string (nullable = true)
 |-- cost_type: string (nullable = true)
 |-- resource_name: string (nullable = true)
 |-- resource_global_name: string (nullable = true)
 |-- component: string (nullable = true)
 |-- SubComponent: string (nullable = false)
 |-- usage_start_time_LOS_ANGELES: timestamp (nullable = true)
 |-- credits_discounts: double (nullable = false)
 |-- credits_promotion: double (nullable = false)
 |-- storage

In [30]:
bucket = "finops-outputs"
spark.conf.set('temporaryGcsBucket', bucket)

In [31]:
GCP_cost_table.write.format('bigquery') \
    .option('table', 'saas-analytics-io.processed.yt1_enh_cost_table_v2') \
    .mode('append') \
    .save()